# Section 3 - Quelques pas plus loin en R 

## <span style="color: steelblue">4. Le package data.table</span> 

## <span style="font-family:Calibri">4.1. Construction d'une data.table</span>
## <span style="font-family:Calibri">4.2. Opérations élémentaires sur data.table</span>
<font size=3 color=grey>
 4.2.1 Selection des lignes et des colonnes<br>
 4.2.2. Renommer et modifier l'ordre des colonnes ou des lignes<br>
 4.2.3. Appliquer des Filtres<br>
 4.2.4. Ajouts de variable et agregations : utiliser le 3em argument "by"<br>
</font>

## <span style="font-family:Calibri">4.3. Opérations avancées</span>  
<font size=3 color=grey>
 4.3.1. Notion de recherche binaire (binary search) avec setkey <br>
 4.3.2. Fonctions spéciales et lapply</span> <br>
 4.3.3. Manipuler des colonnes par référence avec :=  <br>
</font>


<br>


## <span style="color: steelblue">4. Le package data.table</span> 

Le package data.table est né en 2008 sous la plume de Matt Dowle, avec la double ambition de :
 - reduire le temps de programmation grâce a une syntaxe simplifiée
 - reduire le temps machine avec notamment une gestion par clef
 
Le fonctionnement standard d'une requete via data.table est la suivante : 
<b>DT[<font color='steelblue'>i</font>, <font color='green4'>j</font>, <font color='tomato'>by</font>]</b>
que l'on peut lire comme : <br>

<i>"Prends le dataset <b>DT</b>, selectionne les lignes avec <b><font color='steelblue'>i (WHERE)</font></b>, puis calcule <font color='green4'><b>j (SELECT)</b></font> groupé par <font color='tomato'><b>by (GROUP BY)</b></font>"</i>

Notons néanmoins qu'une data.table s'appuie sur la structure de data.frame. On peut donc tout a fait utiliser le langage standard de data.frame pour cet objet.

# <span style="font-family:Calibri">4.1. Construction d'une data.table</span>

Reconstruisons pour l'exemple un dataset factice :


In [ ]:
set.seed(1)
library(data.table)
N <- 100

DT <- data.table(habitat = sample(c("appartement", "maison") , N, replace = T),
                 qualite = sample(LETTERS[(1:5)], N, replace = T),
                 capital = rpois(N, 100))

DT$ligne <- as.integer(rownames(DT) )   # nota : par defaut, le nom de la ligne est son numero. 
                                        #        si deja nommée, utiliser as.integer(rownames())

head(DT)
paste(nrow(DT), "lignes dans l'objet")

class(DT)

<i>Nota : l'affichage de table >60 lignes leve une erreur html sous Jupyter : 
Error in rbindlist(l, use.names, fill, idcol): Item 2 of list input is not a data.frame, data.table or list</i>

# <span style="font-family:Calibri">4.2. Opérations élémentaires sur data.table</span>


### 4.2.1 Selection des lignes et des colonnes

Si la facon generale de selectionner les lignes et colonnes dans une data.table reste tres similaire a celle utilisée pour un data.frame, quelques différences sont a garder en tete.
Dans les exemples ci dessous, on compare les 2 méthodes.
<br>


In [ ]:
DT[(3:5)]                            ## selection des lignes de 3 a 5


In [ ]:
DT[.N]                               ## selection de la derniere ligne

In [ ]:
DT[sample(.N,20)]                    ## on peut du coup utiliser .N pour echantilloner la table

Selection des colonnes : .() est un raccourci pour list()

In [ ]:
head( DT[, .(capital, ligne)]  )      ## selection des colonnes capital et ligne

In [ ]:
DT[c(1, 3, 12), .(capital, ligne)]   ## selection des lignes 1, 3 et 12, et des colonnes capital et ligne

<br><font color=red><b>ATTENTION</b></font> : bien que les data.tables heritent de la classe des data.frame, certaines operations qui marchent naturellement sur un data.frame simple doivent etre ajusté sur une data.table

In [ ]:
## Transformation du data.table en un simple data.frame
DF <- as.data.frame(DT)
class(DF)

In [ ]:

## Selection des colonnes qualité et capital (pour les 3 premieres lignes)
DF[(1:3), c("qualite", "capital")]

In [ ]:

## Idem sur la data.table : ca ne fonctionne pas !
DT[(1:3), c("qualite", "capital")]

In [ ]:

## pour conserver une syntaxe equivalente (avec les noms de colonnes dans un vecteur, entre ""), il faut rajouter with=FALSE
DT[(1:3), c("qualite", "capital"), with = FALSE]

In [ ]:

## L'autre option est d'utiliser la syntaxe data.table (mais qui peut etre moins pratique quand on a stocké 
## les noms de variables dans un vecteur keep par exemple)
DT[(1:3), .(qualite, capital)]

<br>
### 4.2.2. Renommer et modifier l'ordre des colonnes ou des lignes

Comme nous le verrons dans la suite, data.table permet d'effectuer un certain nombre d'operation très rapidement en efectuant les modifications par reference, c'est à dire par l'usage des pointeurs.<br>
Les operateurs suivants fonctionnent pas référence : ils sont donc extremement rapide, y compris sur des datasets volumineux : <br>
- <b>setnames</b>(DT, old, new) permet de remplacer le nom d'une colonne par reference
- <b>setcolorder</b>(DT, ordre) permet de choisir l'ordre des colonnes


In [ ]:
# voici les colonnes disponibles dans notre dataset : 
# (ATTENTION, très important de remettre names(DT) dans un vecteur, sinon "nom" garde un lien avec names(DT) 
#            et on se retrouve a suffixer n fois le nom de colonnes !!)
(nom <- c(names(DT)))

In [ ]:
# Considerons que je souhaite les suffixer par "_MRH" a la façon classique
names(DT) <- paste0(nom, "_MRH")
names(DT)

In [ ]:
# idem mais avec setnames
setnames(DT,  paste0(nom, "_MRH"))
names(DT)

In [ ]:
# Et maintenant on change l'ordre
head(DT, 1)

In [ ]:
setcolorder(DT, order(names(DT)))

In [ ]:
head(DT, 1)

<br>
<br>
La suite de cette section mentionne comment manipuler les data.table avec les fonctionnalités standard du package.
Neanmoins, ces tables pouvant etre manipulées egalement avec la librairie dplyr vue précédemment, la section est mise à disposition pour les cas ou la taille des tables necessite un traitement specifique par clef (binary search) ou par reference (operateur :=)


<br>
### 4.2.3. Appliquer des Filtres
<b><font color=steelblue>Ex.1</b></font> : Selection des appartements

In [ ]:
head( DT[habitat == "appartement"] , n=3)


In [ ]:
# en comparaison, avec un dataframe : 
DF <- as.data.frame(DT)
head (DF[DF$habitat == "appartement", ], n=3)


<br>
<b><font color=steelblue>Ex.2</b></font> : Selection des capitaux entre 100 et 110

In [ ]:
head( DT[capital >=100  & capital <=105] , n=3)


In [ ]:
DT[, .N, by=habitat]


<br>
Pour mieux visualiser les prochaines operations, on reconstruit une data.table de petite dimension : 

<br>
### 4.2.4. Ajouts de variable et agregations : utiliser le 3em argument "by"

data.table permet de facilement integrer des comptages :  

In [ ]:
# Capital total et moyen, sans grouping by
DT[, .(capital.tot = sum(capital), capital.moy = mean(capital))]

Mais grace au 3em argument by, on peut integrer une conditionnalité : 

In [ ]:
# capital moyen par habitat
DT[, .(capital.moy = mean(capital)), by = habitat] 

In [ ]:
set.seed(1)
DT2 <- DT[sample(.N,6)] 
DT2

In [ ]:

# On calcule desormais le capital cumulé par habitat
# A noter que la variable est affichée, mais n'est pas sauvegardée dans la data.table 
DT2[, .(qualite, capital, capital.cum.per_habitat = cumsum(capital)), by = habitat]

In [ ]:

# On peut même pousser le vice jusqu'à utiliser dans l'operande "by" le resultat d'une operation : 
DT2[, .(Count = .N, capital.tot.per.combi = sum(capital)) , by = .(combinaison = paste0(habitat, qualite))]

<br>
# <span style="font-family:Calibri">4.3. Opérations avancées</span>
### 4.3.1. Notion de recherche binaire (binary search) avec setkey
La puissance des data.table est d'offrir la possibilité de construction d'index via l'option <b>setkey</b><br>
Pour autant, nous pourrons en tester la puissance réelle sous RStudio, le container Jupyter ne permettant pas simplement de traiter des grosses données (en tous cas, je n'y arrive pas)


In [ ]:
# sans setkey, voici le moyen de selectionner les liges suivant le filtre mentionné : 
DT[habitat == "appartement" & qualite == "E" ]


In [ ]:
setkey(DT, habitat, qualite)

In [ ]:
# avec setkey : 
DT[.("appartement", "E")]


Exemple de selection de colonne en exploitant la recherche binaire

In [ ]:
head( DT[.("appartement", "E"), .(capital) ] )   
# dans ce cas, on recherche le couple("appartement", "E") de notre clef, et on affiche uniquement capital


<br>
### 4.3.2.  Fonctions spéciales et lapply</span>

Quelques operateurs précalculés peuvent faciliter la lisibilité du code, et permettre d'appliquer en serie des operations : <br>
- <b>.N</b> que nous avons deja vu, et qui donne le nombre de ligne de l'ensemble ou sous ensemble consideré<br>
- <b>.I</b> contient le numéro de la (ou des) lignes consideré<br>
- <b>.SD</b> (pour Subset of Data), contient les valeurs de toutes les colonnes sauf celles specifiées dans le by.
- <b>.SDcols</b> specifie les colonnes de DT incluses dans <b>.SD</b>





Considerons un nouveau dataset, contenant plusieurs colonnes numeriques.

In [ ]:
set.seed(1)
DT <- data.table(x1 = rpois(10, 10), x2 = rpois(10, 5), y = rnorm(10, 20), z = rnorm(10, 5, 1))
DT

Mettons que l'objectif est de rajouter pour chacune de nos colonnes numeriques une nouvelle colonne reproduisant leur moyenne. <br>
Dans une version basique, cela donnerait :

In [ ]:
DT[, .(x1moy = mean(x1), x2moy = mean(x2), ymoy = mean(y), zmoy = mean(z) )]

On peut integrer dans le 2nd operande "y" de data.table une fonction de la famille des apply : lapply, afin d'appliquer a l'ensemble des colonnes contenu dans .SD  (nous verrons plus longuement les fonctions de la famille des apply dans la suite)

In [ ]:
DT[, lapply(.SD, mean)]


Si l'operation ne doit etre pratiquée sur sur un sous ensemble des colonnes, on peut exploiter .SDcols, qui contient les noms des colonnes de .SD

In [ ]:
DT[, lapply(.SD, mean), .SDcols=c("x1", "x2")]

In [ ]:
#.SDcols supportant toute operation lui produisant le vecteur identifiant des colonnes...
DT[, lapply(.SD, mean), .SDcols=(1:2)]

In [ ]:
#.SDcols supportant toute operation lui produisant le vecteur identifiant des colonnes...
DT[, lapply(.SD, mean), .SDcols=paste0("x",1:2)]

In [ ]:
DT[, lapply(.SD, function(x) { c(mean(x), sum(x))} ) ]

<br>
### 4.3.3. Manipuler des colonnes par référence avec := </span>

Utlisation de l'operateur <b>:=</b> 
Possibilité de chaîner les opérations

In [ ]:

# On peut egalement ajouter directement l'element agregé à la data.table : 
head( DT2[, capital.tot := sum(capital)] )

In [ ]:

# Pour ajouter plusieurs nouvelles variables calculées à la data.table entiere, on peut chaîner les operations : 
head( DT2[, capital.med := median(capital)]
         [, capital.TTC := capital*1.206] 
    )

In [ ]:

# La même chose, mais ou les indicateurs sont calculés par habitat pour l'un et par qualité pour l'autre
head( DT2[, capital.sum.per.hab := sum(capital), by = habitat]
         [, capital.sum.per.qual := sum(capital), by = qualite]
    )

In [ ]:

# On peut même pousser le vice jusqu'à utiliser dans l'operande "by" le resultat d'une operation : 
DT2[, capital.sum.per.combi :=sum(capital) , by = .(combinaison = paste0(habitat, qualite))]

Le risque lié a cette operateur := est de se retrouver avec des colonnes dont on souhaite finalement se debarasser.
Une facon de vider une colonne de data.table est de la forcer a NULL (cela marche aussi avec les autres objets en R): 

In [ ]:
DT2[, capital.med:=NULL, ][, capital.tot:=NULL]


<br>
Pour en savoir plus : 
- l'aide mémoire https://s3.amazonaws.com/assets.datacamp.com/img/blog/data+table+cheat+sheet.pdf
- Wiki sur Github : https://github.com/Rdatatable/data.table/wiki
- Vignette sur CRAN : https://cran.r-project.org/web/packages/data.table/vignettes/datatable-intro.pdf
- Un site reprenant l'ensemble du tres complet cours de DataCamp : https://rpubs.com/davoodastaraky/dataTable